In [91]:
!pip install pyttsx3
!pip install scikit_learn
!pip install sentence-transformers
!pip install SpeechRecognition
!pip install flask


import tkinter as tk
from tkinter import ttk, scrolledtext
import webbrowser
import pyttsx3
import json
import threading
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
import numpy as np
import speech_recognition as sr
import math
import random
from flask import Flask, request, jsonify



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:
import os
notebook_dir = os.getcwd()
intents_path = os.path.join(notebook_dir, "intents.json")

try:
    with open(intents_path, "r", encoding="utf-8") as f:
        intent_data = json.load(f)
    print(f" Loaded {len(intent_data)} intents from {intents_path}")
except Exception as e:
    print(f" Could not load intents.json at {intents_path}: {e}")
    intent_data = {
        "hello": "Hi there!",
        "bye": "Goodbye!",
        "tell me a joke": [
            "Why don't scientists trust atoms? Because they make up everything!",
            "Why did the scarecrow win an award? Because he was outstanding in his field!"
        ]
    }

 Loaded 54 intents from d:\College\dl\Jarvis-basic-NLP-personal-assitant-main\intents.json


In [93]:
phrases = list(intent_data.keys())
responses_map = intent_data
labels = phrases

model = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model.encode(phrases)

le = LabelEncoder()
y_encoded = le.fit_transform(labels)

clf = LogisticRegression(max_iter=200)
clf.fit(X_embeddings, y_encoded)

c:\Users\sharm\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,200
,multi_class,'deprecated'


In [94]:
def get_response(user_input: str) -> str:
   

    lower_input = user_input.lower().strip()
   
    if lower_input in ["what time is it", "whats the time", "current time", "tell me the time"]:
        return datetime.now().strftime("It's %I:%M %p right now.")

    if lower_input.startswith("play on youtube"):
        query = lower_input.replace("play on youtube", "").strip()
        if query:
            url = f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"
            webbrowser.open(url)
            return f"Searching and playing '{query}' on YouTube..."
        else:
            return "Please tell me what to play on YouTube."

    if lower_input.startswith("play on spotify"):
        query = lower_input.replace("play on spotify", "").strip()
        if query:
            url = f"https://open.spotify.com/search/{query.replace(' ', '%20')}"
            webbrowser.open(url)
            return f"Searching '{query}' on Spotify..."
        else:
            return "Please tell me what to search on Spotify."

    # FALLBACK TO INTENT CLASSIFIER:
    emb = model.encode([lower_input])
    pred = clf.predict(emb)[0]
    intent = le.inverse_transform([pred])[0]
    resp = responses_map[intent]
    if isinstance(resp, list):
        return random.choice(resp)
    return resp

In [95]:
def speak(text):
    def _run():
        try:
            engine = pyttsx3.init()
            engine.say(text)
            engine.runAndWait()
        except Exception as e:
            print("TTS error:", e)
    threading.Thread(target=_run, daemon=True).start()

def listen_and_respond():
    r = sr.Recognizer()
    with sr.Microphone() as src:
        speak("Listening...")
        print("Listening...")
        audio = r.listen(src, phrase_time_limit=5)
    try:
        voice_text = r.recognize_google(audio)
        print("You said:", voice_text)
        response = get_response(voice_text)
    except Exception as e:
        response = f"Sorry, I couldn't understand: {e}"
    print("Jarvis:", response)
    speak(response)

In [96]:
class JarvisUI:
    def __init__(self, root):
        self.root = root
        self.setup_window()
        self.setup_styles()
        self.create_widgets()

    def setup_window(self):
        self.root.title("J.A.R.V.I.S - Personal Assistant")
        w, h = 800, 600
        self.root.geometry(f"{w}x{h}")
        self.root.configure(bg='#0a0a0a')
        x = (self.root.winfo_screenwidth() // 2) - (w // 2)
        y = (self.root.winfo_screenheight() // 2) - (h // 2)
        self.root.geometry(f"{w}x{h}+{x}+{y}")

    def setup_styles(self):
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('Title.TLabel', background='#0a0a0a', foreground='#00d4ff', font=('Helvetica', 20, 'bold'))
        style.configure('Status.TLabel', background='#0a0a0a', foreground='#00ff41', font=('Helvetica', 10))
        style.configure('Custom.TButton', background='#1a1a1a', foreground='#00d4ff', font=('Helvetica', 12, 'bold'))
        style.map('Custom.TButton', background=[('active', '#2a2a2a'), ('pressed', '#3a3a3a')])

    def create_widgets(self):
        main_frame = tk.Frame(self.root, bg='#0a0a0a')
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)

        header = tk.Frame(main_frame, bg='#0a0a0a')
        header.pack(fill='x', pady=(0,20))
        ttk.Label(header, text="J.A.R.V.I.S", style='Title.TLabel').pack(side='left')
        self.status_label = ttk.Label(header, text="● Online", style='Status.TLabel')
        self.status_label.pack(side='right')

        chat_frame = tk.Frame(main_frame, bg='#1a1a1a', bd=2, relief='sunken')
        chat_frame.pack(fill='both', expand=True, pady=(0,20))
        self.chat = scrolledtext.ScrolledText(chat_frame, bg='#0a0a0a', fg='#00ff41', font=('Consolas',11), wrap=tk.WORD)
        self.chat.pack(fill='both', expand=True)
        for tag, cfg in [('user', {'foreground':'#00d4ff','font':('Consolas',11,'bold')}),
                         ('jarvis', {'foreground':'#00ff41','font':('Consolas',11)}),
                         ('timestamp', {'foreground':'#666666','font':('Consolas',9)})]:
            self.chat.tag_config(tag, **cfg)
        self.chat.config(state='disabled')

        input_frame = tk.Frame(main_frame, bg='#0a0a0a')
        input_frame.pack(fill='x')
        self.entry = tk.Entry(input_frame, bg='#1a1a1a', fg='#00d4ff', font=('Helvetica',12), insertbackground='#00d4ff')
        self.entry.pack(side='left', fill='x', expand=True, padx=(0,10))
        self.entry.bind('<Return>', lambda e: self.on_submit())

        for text, cmd in [("Send", self.on_submit), ("🎤 Voice", self.handle_voice_input), ("Clear", self.clear_chat)]:
            btn = ttk.Button(input_frame, text=text, command=cmd, style='Custom.TButton')
            btn.pack(side='right', padx=(0,10))

        self.add_message('jarvis', "Hello! I'm J.A.R.V.I.S, your personal assistant. How can I help you today?", show_time=True)
        self.entry.focus()

    def add_message(self, sender, message, show_time=False):
        self.chat.config(state='normal')
        if show_time:
            ts = datetime.now().strftime("%H:%M")
            self.chat.insert(tk.END, f"[{ts}] ", 'timestamp')
        tag = 'user' if sender=='user' else 'jarvis'
        self.chat.insert(tk.END, f"{sender.title()}: ", tag)
        self.chat.insert(tk.END, message + "\n\n")
        self.chat.config(state='disabled')
        self.chat.see(tk.END)

    def on_submit(self):
        text = self.entry.get().strip()
        if not text: return
        self.entry.delete(0, tk.END)
        self.add_message('user', text, show_time=True)
        self.status_label.config(text="● Processing...")
        self.root.update()
        resp = get_response(text)
        self.add_message('jarvis', resp, show_time=True)
        speak(resp)
        self.status_label.config(text="● Online")
        self.entry.focus()

    def handle_voice_input(self):
        voice = listen_voice_input()
        if voice:
            self.add_message('user', voice, show_time=True)
            resp = get_response(voice)
            self.add_message('jarvis', resp, show_time=True)
            speak(resp)
        self.status_label.config(text="● Online")

    def clear_chat(self):
        self.chat.config(state='normal')
        self.chat.delete(1.0, tk.END)
        self.chat.config(state='disabled')
        self.add_message('jarvis', "Chat cleared. How can I help you?", show_time=True)


In [97]:
if __name__ == '__main__':
    root = tk.Tk()
    app = JarvisUI(root)
    root.mainloop()
